##Installing dependencies and imports

In [ ]:
!pip install -q gradio==3.50.2 pypdf2==3.0.1 python-docx==0.8.11 faiss-cpu==1.7.4 \
    transformers==4.30.2 torch==2.0.1 sentence-transformers==2.2.2 \
    langchain langchain-community ctransformers python-jose

# Import required libraries
import os
import gradio as gr
from typing import List, Dict
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import faiss
import PyPDF2
from docx import Document
import pickle
from langchain_community.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import faiss
import numpy as np

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 77.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q redis python-jose

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.5/261.5 kB 3.9 MB/s eta 0:00:00


##Setting up Spark

In [ ]:
import json
import os

from google.colab import files

if 'spark_nlp_for_healthcare_spark_ocr_9813.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_nlp_for_healthcare_spark_ocr_9813.json')

with open('spark_nlp_for_healthcare_spark_ocr_9813.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

license_keys.keys()
license_keys['JSL_VERSION']
license_keys['PUBLIC_VERSION']

# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.4.0 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

Saving spark_nlp_for_healthcare_spark_ocr_9813.json to spark_nlp_for_healthcare_spark_ocr_9813.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.6/626.6 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.9/538.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.0 MB/s eta 0:00:00


In [ ]:
#Setting up spark
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pipeline_tracer import PipelineTracer
from sparknlp_jsl.pipeline_output_parser import PipelineOutputParser

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel

import json
import pandas as pd
pd.set_option('display.max_colwidth', 200)

from pyspark.sql.functions import col, explode, lit

from tqdm import tqdm
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"10G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 5.5.1
Spark NLP_JSL Version : 5.5.1


In [ ]:
#Installing pdf and doc readers.
! pip install pymupdf python-docx pyspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 79.5 MB/s eta 0:00:00


##LLM

In [ ]:
! pip install transformers
! pip install torch
! pip install keras

In [ ]:
#Setting up the LLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Encode the input text


def chat_with_claude(prompt):
  inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
  # tokenizer.pad_token = tokenizer.eos_token
  inputs["pad_token_id"] = tokenizer.pad_token_id
  # Generate a response
  # outputs = model.generate(inputs['input_ids'], max_length=500, num_return_sequences=1)
  outputs = model.generate(inputs['input_ids'], attention_mask=inputs['attention_mask'], pad_token_id=inputs['pad_token_id'], max_length=200)

  # Decode the output
  # print(outputs[0])
  response = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print("printing response")
  print(response)
  return response


In [ ]:
#Huggngface token
! export HF_TOKEN=******** #Insert huggingface access token here

In [ ]:
#Creating the prompt
def create_prompt(context, question):
    return f"""Use the following context to answer the question. make the answer concise to 10 words or less than 10 words
    If you don't find the answer in the context, say "I don't have enough information to answer that."

    Context: {context}

    Question: {question}"""

##Vector DB

In [ ]:
#Similarity Search
def search_similar(faiss_index, question_embedding, file_text, k):
    print("calling search similar function")
    # return search_faiss(question_embedding, file_text ,k=3)
    query_embedding = np.array(question_embedding).reshape(1, -1).astype('float32')
    distances, indices = faiss_index.search(query_embedding, k)
    results = [(file_text[idx], distances[0][i]) for i, idx in enumerate(indices[0])]
    return results

In [ ]:
#FAISS Index
def create_faiss_index(df):
    dimension = len(df.iloc[0]['embeddings'])  # Assuming all embeddings are of the same length
    faiss_index = faiss.IndexFlatL2(dimension)
    print("faiss index created")
    return faiss_index

##Doc processing

In [ ]:
#Doc processing using Spark Pipeline
from pyspark.sql.types import StructType, StructField, StringType
def process_file(file_text):
    schema = StructType([
        StructField("text", StringType(), True)
    ])
    result_df = spark.createDataFrame(file_text, schema)
    # import sparknlp


    # # Define the pipeline
    document_assembler = DocumentAssembler().setInputCol("text").setOutputCol("document")
    sentence_detector = SentenceDetector().setInputCols("document").setOutputCol("sentence")
    embeddings = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en', 'clinical/models').setInputCols(["sentence"]).setOutputCol("sentence_embeddings")

    # # Combine into a pipeline
    pipeline = Pipeline(stages=[document_assembler, sentence_detector, embeddings])

    # # Example text
    data = spark.createDataFrame(file_text, schema)

    # # Apply the pipeline
    model = pipeline.fit(data)
    result = model.transform(data).cache()
    resultWithSize = result.selectExpr(
        "sentence.result as result",
        "sentence_embeddings.embeddings as embeddings"
    )
    res2 = resultWithSize.selectExpr(
        "arrays_zip(result, embeddings) as result_embeddings"
    )
    final = res2.select(explode("result_embeddings").alias("name_age_pair"))
    final_max = final.selectExpr(
        "name_age_pair.result as result",
        "name_age_pair.embeddings as embeddings"
    ).toPandas()
    return final_max

In [ ]:
#Test df to check chunking and embedding.
df = process_file(file_text=[["this is sentence one. this is sentence two .this is sentence three."]])
df

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


,result,embeddings
0,this is sentence one.,"[0.35628798604011536, -0.48671117424964905, 0.0836561992764473, 0.22219394147396088, 1.161763310432434, 0.2215859740972519, -0.18069948256015778, 0.354088693857193, -0.08967858552932739, 0.6732552..."
1,this is sentence two .,"[0.14464564621448517, -0.25935453176498413, -0.6000803112983704, 0.629290759563446, 1.0368129014968872, 0.11163471639156342, -0.22896036505699158, -0.014659400098025799, 1.1230627298355103, 0.1673..."
2,this is sentence three.,"[0.7257365584373474, -0.20896494388580322, -0.7927187085151672, 0.28782516717910767, 1.0485575199127197, -0.04332759603857994, -0.04275362938642502, 0.7431215047836304, -0.30505460500717163, 0.648..."


##Caching and Auth

In [ ]:
#Caching
# Simple in-memory cache for Colab
cache = {}

# To get a value from the cache
cache_get = cache.get

# To set a value in the cache
cache_set = lambda key, value: cache.update({key: value})

# To clear the cache
cache_clear = cache.clear

In [ ]:
#Authenticaton
import jwt
import datetime

# Secret key for encoding/decoding tokens (use a secure key in production)
SECRET_KEY = "your-secret-key"

# Check user credentials
check_credentials = lambda username, password: username == "username" and password == "password"

# Create a JWT token
create_token = lambda username: jwt.encode(
    {"username": username, "exp": datetime.datetime.utcnow() + datetime.timedelta(hours=1)},
    SECRET_KEY,
    algorithm="HS256"
)

# Verify a JWT token
def verify_token(token: str) -> bool:
    try:
        jwt.decode(token, SECRET_KEY, algorithms=["HS256"])
        return True
    except:
        return False


##UI Integration

In [ ]:
#Using gradio for UI integration and connection with the models
import gradio as gr
from docx import Document
import fitz
faiss_index = ''
texts = []
# Function to handle file upload
def handle_file_upload(file):
    if file is None:
        return "Please upload a file first."

    try:
        # Read file content based on file type
        if file.name.endswith('.pdf'):
            print("line 13 detected pdf")
            # file_text = fitz.open(file.name).get_text()
            pdf_doc = fitz.open(file.name)
            file_text = ""
            for page in pdf_doc:
                print("line 18 looping over pages")
                file_text += page.get_text()
            pdf_doc.close()
        elif file.name.endswith('.docx'):
            doc = Document(file.name)
            file_text = "\n".join([p.text for p in doc.paragraphs])
        else:
            return "Unsupported file format. Please upload PDF or DOCX files."

        # Process file content
        # print(type(file_text))
        if isinstance(file_text, str):
          data = [[file_text]]
        result = process_file(data)
        print(f"length of embeddings or texts {len(result['embeddings'].tolist())}")
        print("processed file text")

        global faiss_index
        faiss_index = create_faiss_index(result)

        faiss_index.add(np.array(result['embeddings'].tolist()).astype('float32'))
        print(f"type of {type(faiss_index)} ")
        print("embeddings recieved")

        # Add texts to global storage
        # new_texts = get_texts(result)
        # print(new_texts)
        # texts.extend(new_texts)
        global texts
        texts = result['result'].to_list()
        print("Text to global storage done")
        return {"status": 200, "body": "File processed successfully!"}

    except Exception as e:
        return f"Error processing file: {str(e)}"

# Function to handle question answering
def handle_question(question):
    if not question:
        return "Please enter a question."

    # Check cache
    # cache_key = hash_key(question)
    # cached_answer = cache.get(cache_key)
    # if cached_answer:
    #     return f"{cached_answer} (Cached)"

    try:
        # Get question embedding
        if isinstance(question, str):
          data = [[question]]
        question_result = process_file(data)
        question_embedding = question_result['embeddings'].to_list()
        print("question embeddings generated")
        # Search similar documents
        global faiss_index
        global texts
        indices = search_similar(faiss_index, question_embedding, texts, k=10)
        print(indices)
        relevant_chunks = [i[0] for i in indices]
        print(relevant_chunks)

        print("relevant_chunks")

        # Create prompt and get answer
        context = "\n".join(relevant_chunks)
        prompt = create_prompt(context, question)
        # print(prompt)
        answer = chat_with_claude(prompt)

        # Cache answer
        # cache[cache_key] = answer

        return answer

    except Exception as e:
        return f"Error generating answer: {str(e)}"

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Medical Document QA System")

    with gr.Row():
        file_input = gr.File(
            label="Upload PDF or DOCX file",
            file_types=[".pdf", ".docx"]
        )
        process_button = gr.Button("Process File")
        output_text = gr.Textbox(label="Processing Status")

    with gr.Row():
        question_input = gr.Textbox(label="Ask a question about the documents")
        submit_button = gr.Button("Get Answer")

    answer_output = gr.Textbox(label="Answer")

    process_button.click(
        fn=handle_file_upload,
        inputs=[file_input],
        outputs=output_text,
        api_name="process_file",
        show_progress=True
    )

    submit_button.click(
        fn=handle_question,
        inputs=[question_input],
        outputs=answer_output
    )

# Launch the interface
demo.queue(concurrency_count=1, max_size=20).launch(
    debug=True,
    share=True,
    max_threads=1  # Controls concurrent processing
)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://df04c2b959059e4785.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


line 13 detected pdf
line 18 looping over pages
line 18 looping over pages
line 18 looping over pages
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
length of embeddings or texts 36
processed file text
faiss index created
type of <class 'faiss.swigfaiss_avx2.IndexFlatL2'> 
embeddings recieved
Text to global storage done
sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
question embeddings generated
calling search similar function
[('Mental health exists on a continuum, or range: \nfrom positive, healthy functioning at one end \nthrough to severe symptoms of mental health \nconditions at the other.', 162.40167), ('Like physical health, mental health is not fixed.', 167.69418), ('•\t\nWhat will you use next time you speak about mental health conditions?', 177.55908), ('UNDERSTANDING MENTAL HEALTH\nWHAT IS MENTAL HEALTH?', 189.18271), ('If you are concerned about your